In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
from matplotlib import rc

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig_dir = "figures"
os.makedirs(fig_dir, exist_ok=True)

adv_out_dir = os.path.join(fig_dir, "adversarial")
os.makedirs(adv_out_dir, exist_ok=True)

In [ ]:
from avapi.carla import CarlaScenesManager

data_root = "/data/shared/CARLA/multi-agent-intersection/"
CSM = CarlaScenesManager(data_dir=data_root, split="train")
idx_scene = 3
CDM = CSM.get_scene_dataset_by_index(idx_scene)

## Apply an adversary model on top of the raw data

Show temporal evolution

In [ ]:
# get a lidar point cloud
idx_frame = 60
frames = CDM.get_frames(sensor="lidar-0", agent=0)
agent0 = CDM.get_agent(frame=frames[idx_frame], agent=0)
pc = CDM.get_lidar(frame=frames[idx_frame], sensor="lidar-0", agent=0)
img_front = CDM.get_image(frame=frames[idx_frame], sensor="camera-0", agent=0)
img_back = CDM.get_image(frame=frames[idx_frame], sensor="camera-1", agent=0)

In [ ]:
from avsec.multi_agent.adversary import AdversaryModel
from avsec.multi_agent.manifest import (
    FalsePositiveManifest,
    FalseNegativeManifest,
    TranslationManifest,
)
from avsec.multi_agent.propagation import MarkovPropagator


manifests = [
    FalsePositiveManifest(seed=0, exact_select=2),
    FalseNegativeManifest(seed=0, exact_select=2),
    TranslationManifest(seed=0, exact_select=2),
]

# show different manifest behaviors
for manifest in manifests:
    # set up the adversary
    adversary = AdversaryModel(
        propagator=MarkovPropagator(),
        manifest=manifest,
        dt_init=2,
        dt_reset=20,
        seed=0,
    )

    # loop over frames for experiment
    objs_all = []
    for frame in CDM.get_frames(sensor="lidar-0", agent=0)[4:]:
        # load the next set of data
        agent = CDM.get_agent(frame=frame, agent=0)
        objs_agent = CDM.get_objects(frame=frame, sensor="lidar-0", agent=0)

        # run the adversary
        objs_out = adversary(objs_agent, fov=None, reference=agent.as_reference())

        # save the objects
        objs_all.append(objs_out)

    # make the trajectory image
    pass